In [35]:
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils

## Data

In [24]:
fn = 'data/shakespeare.txt'
sentences = []
all_chars = []
with open(fn, 'r') as f:
    for line in f:
        if len(line.strip()) > 3:
            if line[:2] == '  ': line = line[2:]
            sentences.append(line.lower())
            all_chars += list(line.lower())

In [26]:
# Create vocabulary.
chars = sorted(list(set(all_chars)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(all_chars)
n_vocab = len(chars)
print('Total characters: %d' % n_chars)
print('Total vocab: %d' % n_vocab)

Total characters: 93673
Total vocab: 38


In [27]:
# Construct dataset.
seq_len = 40
step_size = 3
dataX = []
dataY = []
for i in range(0, n_chars - seq_len, step_size):
    seq_in = all_chars[i: i + seq_len]
    seq_out = all_chars[i + seq_len]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print('Total patterns: %d' % n_patterns)

Total patterns: 31211


In [28]:
X = np.reshape(dataX, (n_patterns, seq_len, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

## Train model

In [45]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_6 (Dense)              (None, 38)                4902      
Total params: 71,462
Trainable params: 71,462
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [55]:
model.fit(X, y, epochs=165, batch_size=128, initial_epoch=160)

Epoch 161/165
31211/31211 [==============================] - 8s 251us/step - loss: 1.3361 - accuracy: 0.5817
Epoch 162/165
31211/31211 [==============================] - 8s 249us/step - loss: 1.3156 - accuracy: 0.5864
Epoch 163/165
31211/31211 [==============================] - 8s 249us/step - loss: 1.2686 - accuracy: 0.6014
Epoch 164/165
31211/31211 [==============================] - 8s 249us/step - loss: 1.2399 - accuracy: 0.6142
Epoch 165/165
31211/31211 [==============================] - 8s 249us/step - loss: 1.3114 - accuracy: 0.5867


## Generate text

In [8]:
def sample(preds, temperature=1.0):
    # Helper function to sample from a probability array
    # Source: https://keras.io/examples/lstm_text_generation/
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)

In [57]:
seed = "shall i compare thee to a summer's day?\n"
pattern = [char_to_int[char] for char in seed]
output_seq = pattern

In [58]:
tmp = 1.0
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #index = sample(prediction, temperature=tmp)
    index = np.argmax(prediction)
    pattern.append(index)
    pattern = pattern[1:]
    output_seq.append(index)

In [59]:
output = [int_to_char[i] for i in output_seq]
output = ''.join(output)
print(output)

shall i compare thee to a summer's day?
ooy nowe nend eye sey suellc shat pnoa iooe to get,
toeeter on turm cnmc siou drt fort theer brent,
when io at thou some iey seyslnt somlld,
that ho an bnl-
bns the parter puoz ouecrire mfnd thu bioigr,
what tomrdr each of madeid ooog aoteer,
and then f sureeg'c cuente the srire coon,
thet ttimte that ier booeer thanl by their stete,
what somrdr me toae in torr in the rhnwe.
thoce tome supaegss of deauty's fartane frowt,
and to move shall whet i davtet to mh.
that thou sha teysrec toatl of toy minht sheng,
aut thou altntit than, then thou sha dvis,
the earte coou thou drt thy siof, and yhuh mi,
what thou sharo iiv stbtt wot,srmd somn miare,
and shat bnue ou lassr aavnarg yourg,
and shat in tore io torl aea bm b foenr,
aut whete thou art bnth mu cupe ou lens,
and bol hyer the sreer toure thoug thel pooiige slgte,
and thenefore ane then e sapenuer comh,
to tie poeer's fars frcehng hand dv theer ocys,
ay as i hou feytery dremnid fyely drol.
and aol 

In [56]:
model.evaluate(X, y)

31211/31211 [==============================] - 5s 163us/step


[1.2009560575030256, 0.628304123878479]